In [7]:
import pandas as pd
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [98]:
df=pd.read_csv('/Users/vedantgupta/Documents/summer project/data-analysis-with-technical-indicators-2/final/Data_imports/TITAN.NS(close).csv')
df

,Date,TITAN.NS(close)
0,2011-01-03,172.213837
1,2011-01-04,170.863907
2,2011-01-05,170.083847
3,2011-01-06,167.194824
4,2011-01-07,159.387192
...,...,...
3199,2023-12-22,3627.350098
3200,2023-12-26,3656.699951
3201,2023-12-27,3689.250000
3202,2023-12-28,3715.100098


In [99]:
df.dtypes

Date                object
TITAN.NS(close)    float64
dtype: object

In [100]:
#parameters
n = 12
m = 26
t = 9

In [101]:
df.loc[:,'EMAn'] = df['TITAN.NS(close)'].ewm(span=n, adjust=False).mean()

df.loc[:,'EMAm'] = df['TITAN.NS(close)'].ewm(span=m, adjust=False).mean()


In [102]:
for i in range(n):
    df.loc[i,'EMAn'] = None
for i in range(m):
    df.loc[i,'EMAm'] = None
    

In [103]:
df['MACD'] = df['EMAn'] - df['EMAm']
df['Signal Line'] = df['MACD'].ewm(span=t, adjust=False).mean()
df

,Date,TITAN.NS(close),EMAn,EMAm,MACD,Signal Line
0,2011-01-03,172.213837,NaN,NaN,NaN,NaN
1,2011-01-04,170.863907,NaN,NaN,NaN,NaN
2,2011-01-05,170.083847,NaN,NaN,NaN,NaN
3,2011-01-06,167.194824,NaN,NaN,NaN,NaN
4,2011-01-07,159.387192,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
3199,2023-12-22,3627.350098,3579.676120,3513.063006,66.613115,73.847626
3200,2023-12-26,3656.699951,3591.525941,3523.702780,67.823161,72.642733
3201,2023-12-27,3689.250000,3606.560411,3535.965537,70.594875,72.233161
3202,2023-12-28,3715.100098,3623.258825,3549.234763,74.024061,72.591341


In [104]:
trade_sig=np.zeros([len(df),1],dtype=float)

for i in range(m+1,len(df)):
    if df.loc[i,'MACD'] > df.loc[i,'Signal Line'] and df.loc[i-1,'MACD'] < df.loc[i-1,'Signal Line']:
        trade_sig[i] = 1
    if df.loc[i,'MACD'] < df.loc[i,'Signal Line'] and df.loc[i-1,'MACD'] > df.loc[i-1,'Signal Line']:
        trade_sig[i] = -1

In [105]:
trade_sig=trade_sig.reshape(len(trade_sig),1)

In [106]:
trade_sig.shape

(3204, 1)

In [107]:
df_numpy=df.to_numpy()

In [108]:
final=np.column_stack((df_numpy,trade_sig))

In [109]:
final_df=pd.DataFrame(final,columns=['Date','Close','EMAn','EMAm','MACD','Signal Line','Trade Signal'])

In [110]:
final_df.drop(["EMAn","EMAm","MACD","Signal Line"],axis=1,inplace=True)

In [111]:
final_df['Close'] = final_df['Close'].astype(float) 
final_df['Trade Signal'] = final_df['Trade Signal'].astype(float)

In [112]:
final_df.dtypes

Date             object
Close           float64
Trade Signal    float64
dtype: object

In [113]:
final_df.to_csv('/Users/vedantgupta/Documents/summer project/data-analysis-with-technical-indicators-2/final/trading_signals/titan_ti_ts.csv',index=False)